<a href="https://colab.research.google.com/github/UOS-COMP6252/public/blob/main/lecture8/music-classification-pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="text-align: center;">COMP6252 Deep Learning Technologies</h1>
<h2 style="text-align: center;"> Transfer learning- music genre classification</h2>

In [ ]:
import sys,os
from pathlib import Path
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  from google.colab import files
  if not Path('/content/gtzan-dataset-music-genre-classification.zip').is_file(): 
    file=files.upload() # upload the saved kaggle.json
    #### the token can be obtained from your kaggle account by going to settings
    #### in the middle of the page under API there is a create new token
    #### there is also explanation on how to do it
    %mkdir /root/.kaggle
    %mv kaggle.json  /root/.kaggle
    %kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification
    %unzip -q /content/gtzan-dataset-music-genre-classification.zip
    %pip install timm==0.4.12
    # get_ipython().system('mkdir /root/.kaggle # on colab you are use root')
    # get_ipython().system('mv kaggle.json  /root/.kaggle')
    # get_ipython().system('kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification')
    # get_ipython().system('unzip -q /content/gtzan-dataset-music-genre-classification.zip')
    ####### for some reason the new timm versions are not working
    # get_ipython().system('pip install timm==0.4.12')
  data_path="./Data/images_original"
elif os.environ.get('KAGGLE_KERNEL_RUN_TYPE') is not None:
  %pip install timm==0.4.12 >/dev/null 2>&1
  %pip install comet-ml >/dev/null 2>&1
  # get_ipython().system('pip install comet-ml >/dev/null 2>&1')
  # get_ipython().system('pip install timm==0.4.12')
  ##### make sure that you have added the dataset
  data_path="/kaggle/input/gtzan-dataset-music-genre-classification/Data/images_original"
else:
  #data_path=os.path.join(os.path.expanduser("~"),"Data/images_original")
  data_path="Data/images_original"
  if not os.path.isdir(data_path):
    !kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification
    !unzip -q gtzan-dataset-music-genre-classification.zip


In [ ]:
import comet_ml
from comet_ml import Experiment
import lightning as L
from lightning.pytorch import seed_everything
from lightning.pytorch.loggers import CSVLogger,CometLogger
from lightning.pytorch.callbacks import ModelCheckpoint,Callback

# %% In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset,DataLoader,random_split
import matplotlib.pyplot as plt
import copy
import torchmetrics
from tqdm import tqdm
import timm
from transformers import AutoModelForImageClassification

In [ ]:
comet_ml.init(project_name="music-classification")
experiment=Experiment()
hub='timm' # option 'timm','torch','hface'
hub='torch'
hub='hface'
#generic_name='vit'
generic_name='resnet18'
model_name={'resnet18':'resnet18','resnet50':'resnet50','vit':'vit_base_patch32_224' if hub=='timm' else 'vit_b_32'}

hyper_params = {'hub':hub,"batch_size": 64, "num_epochs": 50, "learning_rate": 0.001,"momentum":0.9,"num_workers":2,
                "model_name":model_name[generic_name],'pretrained':True,'use_aug':True}
experiment.log_parameters(hyper_params)

experiment.set_name(hub+'-'+model_name[generic_name])
seed_everything(123, workers=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset=torchvision.datasets.ImageFolder(data_path)
num_classes=len(dataset.classes)


In [ ]:
model=Model(generic_name,num_classes=num_classes)
model=model.to(device)

In [ ]:
if hub=='timm':
    mean=list(model.cfg()['mean'])
    std=list(model.cfg()['std'])
else:
    mean= [0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]

In [ ]:

class CustomDataset(Dataset):
    def __init__(self,subset,transform=None):
        self.subset=subset
        self.transform=transform
    def __getitem__(self,idx):
        x,y=self.subset[idx]
        if self.transform:
            x=self.transform(x)
        return x,y
    def __len__(self):
        return len(self.subset)

data_transforms = {
     'train':  transforms.Compose([ transforms.TrivialAugmentWide(),
                                   transforms.CenterCrop(224),transforms.ToTensor(),
                                   transforms.Normalize(mean, std)]) if hyper_params['use_aug'] else
    transforms.Compose([
                                   transforms.CenterCrop(224),transforms.ToTensor(),
                                   transforms.Normalize(mean, std)])
    ,
  
    'val': transforms.Compose([transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    
    'test': transforms.Compose([transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
}

datasets={}
datasets['train'],datasets['val'],datasets['test']=random_split(dataset,lengths=[0.7,0.2,0.1])
image_datasets = {x: CustomDataset(datasets[x],data_transforms[x])
                  for x in ['train', 'val','test']}
dataset_sizes={x:len(image_datasets[x]) for x in ['train','val','test']}
dataloaders={x:DataLoader(image_datasets[x], batch_size=hyper_params['batch_size'],
                                             shuffle=True if x=='train' else False, num_workers=hyper_params["num_workers"])
                                             for x in ['train','val','test']}




In [ ]:
timm_model=timm.create_model('resnet18',pretrained=True,num_classes=num_classes)  
weights_enum=torch.hub.load('pytorch/vision','get_model_weights','resnet18')
torch_model=torch.hub.load('pytorch/vision','resnet18',weights=weights_enum)
hface_model=AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")
timm_modules=[m for m in timm_model.named_modules()]
torch_modules=[m for m in torch_model.named_modules()]
hface_modules=[m for m in hface_model.named_modules()]
print(timm_modules[-1])
print(torch_modules[-1])
print(hface_modules[-1])

In [ ]:

class Model(nn.Module):
    def __init__(self,generic_name,num_classes):
        super().__init__()
        # self.model=timm.create_model(model_name=model_name,pretrained=True)
        # for p in self.model.parameters():
        #     p.requires_grad=False   
        if hub=='timm':
            self.model=timm.create_model(model_name=model_name[generic_name],num_classes=num_classes,pretrained=hyper_params['pretrained'])
        if hub=='torch':
            if hyper_params['pretrained']:
                weights_enum=torch.hub.load('pytorch/vision','get_model_weights',model_name[generic_name])
                self.model=torch.hub.load('pytorch/vision',model_name[generic_name],weights=weights_enum)
            else:
                self.model=torch.hub.load('pytorch/vision',model_name[generic_name],weights=None)
            if generic_name.startswith('vit'):
              self.model.heads=nn.Linear(self.model.heads[0].in_features,num_classes)
            else:
              self.model.fc=nn.Linear(self.model.fc.in_features,num_classes)
        if hub=='hface':
            self.model=AutoModelForImageClassification.from_pretrained('microsoft/resnet-18')
            self.model.classifier[1]=nn.Linear(in_features=512,out_features=num_classes)
    
    def forward(self,x):
        return self.model(x).logits if hub=='hface' else self.model(x)
    def cfg(self):
        return self.model.default_cfg

In [ ]:
def test(model, criterion,loader):
    with torch.no_grad():
        model.eval()   
        running_loss = 0.0        
        test_acc=torchmetrics.Accuracy('multiclass',num_classes=num_classes).to(device)
        confmat=torchmetrics.ConfusionMatrix(task='multiclass',num_classes=num_classes).to(device)
        for inputs, labels in loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                confmat.update(preds,labels)
                batch_acc=test_acc(preds,labels.data)
            
        total_acc=test_acc.compute()
       
    return total_acc,confmat

In [ ]:
def validate(model, criterion,loader):
    with torch.no_grad():
        model.eval()   
        running_loss = 0.0        
        val_acc=torchmetrics.Accuracy('multiclass',num_classes=num_classes).to(device)
        for inputs, labels in loader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * hyper_params['batch_size']
                batch_acc=val_acc(preds,labels.data)
            
        epoch_loss = running_loss / (len(loader)*hyper_params['batch_size'])
        epoch_acc=val_acc.compute()
       
    return epoch_loss,epoch_acc

In [ ]:
def train_model(model, criterion, optimizer,scheduler=None, num_epochs=100):
    for epoch in range(hyper_params['num_epochs']):
        
        model.train()  
        running_loss=0.
        best_model_wts = copy.deepcopy(model.state_dict())
        best_acc = 0.0
      
        train_acc=torchmetrics.Accuracy('multiclass',num_classes=num_classes).to(device)
        loop=tqdm(dataloaders['train'])
        loop.set_description(f"Epoch [{epoch+1}/{hyper_params['num_epochs']}]")

        for inputs, labels in loop:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            batch_acc=train_acc(preds,labels.data)

            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss =0.9*running_loss+0.1*loss.item() 
            if epoch>0:
                 loop.set_postfix(loss=running_loss,t_acc=epoch_acc.item(),val_acc=v_acc.item())
            else:
                 loop.set_postfix(loss=running_loss)
       

        epoch_loss = running_loss / dataset_sizes['train']
        epoch_acc=train_acc.compute()        
        experiment.log_metric(epoch_loss,epoch)
        v_loss,v_acc=validate(model,criterion,dataloaders['val'])
        experiment.log_metrics({"train_loss":epoch_loss,"train_acc":epoch_acc},epoch=epoch)

        experiment.log_metrics({"val_loss":v_loss,"val_acc":v_acc},epoch=epoch)

        if  v_acc > best_acc:
                best_acc = v_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        train_acc.reset()
    ## training is done. Return the model with the best validation accuracy
    model.load_state_dict(best_model_wts)
    return model

In [ ]:


model=model.to(device)
optimizer=optim.SGD(model.parameters(),lr=hyper_params['learning_rate'],momentum=hyper_params['momentum'])
#optimizer=optim.Adam(model.parameters(),lr=0.0001)
criterion=nn.functional.cross_entropy

model=train_model(model=model,criterion=criterion,optimizer=optimizer,scheduler=None,num_epochs=hyper_params['num_epochs'])


test_acc,confmat=test(model,criterion,dataloaders['test'])
print(test_acc.item())
experiment.log_metric("test_acc",test_acc.item())

print(test_acc.item())

mat=confmat.compute().cpu().numpy()
experiment.log_confusion_matrix(matrix=mat,labels=dataset.classes)
experiment.end()